<a href="https://colab.research.google.com/github/LaviJ/Cryptocurrency-Analysis/blob/main/Machine%20Learning/hourly_model_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Forecasting Cryptocurrency
# Data Analysis and Visualization
# Group 7

# TensorFlow model prediction routine

import os
import datetime
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import numpy as np
import pandas as pd
import seaborn as sns

symbols=['BTC','ETH','XRP','USDT','ADA','XMR','XLM','BUSD','LTC','DOGE']
modeltypes=['lstm','bidir','feedback']
testordinal='4'
for symbol in symbols:
  for modeltype in modeltypes:
    # Declare the dataframes
    train_df = val_df = test_df = realdf = df = pd.DataFrame()

    # Load the trained tensorflow model
    # TODO: make coin variable
    themodel = keras.models.load_model('hourlyCryptoModels/'+symbol+modeltype+'_model')

    # Use Keras to get the data file
    path = tf.keras.utils.get_file(
        origin='https://raw.githubusercontent.com/LaviJ/Cryptocurrency-Analysis/main/Data/Hourly/Test'+testordinal+'/test'+testordinal+'_'+symbol+'.csv',
        fname='test'+testordinal+'_'+symbol+'.csv')
    csv_path, _ = os.path.splitext(path)

    # Read data into dataframe
    df = pd.read_csv(csv_path+".csv")

    # Use Keras to get the *real* data file
    path = tf.keras.utils.get_file(
        origin='https://raw.githubusercontent.com/LaviJ/Cryptocurrency-Analysis/main/Data/Hourly/Test'+testordinal+'/Results/Analysis/real'+testordinal+'_'+symbol+'.csv',
        fname='real'+testordinal+'_'+symbol+'.csv')
    csv_path, _ = os.path.splitext(path)

    # Read real data into dataframe
    realdf = pd.read_csv(csv_path+".csv")

    # Drop some columns
    df = df.drop('conversionType', axis=1)
    df = df.drop('conversionSymbol', axis=1)
    df = df.drop('volumeto', axis=1)
    df = df.drop('volumefrom', axis=1)
    df['mid'] = (df['high'] + df['low'])/2
    df = df.drop('open', axis=1)
    df = df.drop('high', axis=1)
    df = df.drop('low', axis=1)

    # Drop some columns
    realdf = realdf.drop('conversionType', axis=1)
    realdf = realdf.drop('conversionSymbol', axis=1)
    realdf = realdf.drop('volumeto', axis=1)
    realdf = realdf.drop('volumefrom', axis=1)
    realdf['mid'] = (realdf['high'] + realdf['low'])/2
    realdf = realdf.drop('open', axis=1)
    realdf = realdf.drop('high', axis=1)
    realdf = realdf.drop('low', axis=1)

    # Drop last row
    df = df[:-1]
    realdf = realdf[:-1]

    last = df.iloc[0]['close']
    print(last)

    # Move the timestamps into an array
    date_time =[0]
    date_time_real =[0]
    date_time = df.pop('time')
    date_time_real = realdf.pop('time')
    firsttime = str(pd.to_datetime(date_time_real.iloc[-1],unit='s'))

    column_indices = {name: i for i, name in enumerate(df.columns)}

    n = len(df)
    train_df = df[0:int(n*0.7)]
    val_df = df[int(n*0.7):int(n*0.9)]
    test_df = df[int(n*0.9):]

    num_features = df.shape[1]

    MAX_EPOCHS = 11
    OUT_STEPS = 96

    # Show the input data
    #df.plot(legend=True, figsize=(5,3), xlabel='Minutes since '+firsttime+' UTC', ylabel='$ (USD)', colormap='coolwarm', title=symbol+': 96-hour Input Data', lw=3)
    #plt.savefig('pngs/'+symbol+modeltype+'in.png',bbox_inches='tight',dpi=150)

    # Adapt input array dimensions to fit trained model shape [None, 96, 8] 96 points of 8 dimensional data (2 price points, 6 periodic waveforms)
    
    # normalize dataset with MinMaxScaler
 
    scaler = MinMaxScaler(feature_range=(-1,1))
    
    df['close']  = scaler.fit_transform(np.array(df['close']).reshape(-1, 1)) 
    df['mid']  = scaler.fit_transform(np.array(df['mid']).reshape(-1, 1)) 
    df['Daily Sine']=df['Daily Cosine']=df['Quarterly Sine']=df['Quaterly Cosine']= df['Annual Sine']=df['Annual Cosine']=last
   
    answer=pd.DataFrame()
    # Run feedback model prediction on input data

    first = np.zeros(96)
    for iter in range(96):
      x = tf.convert_to_tensor( df[:] )
      x = tf.expand_dims(x, 0, name="inputs")
      x = tf.ensure_shape(x, [1, 96, 8])
      with tf.device('/device:GPU:0'):
        answer = pd.DataFrame(themodel.predict(x,False)[0])

    # Put the answer into a dataframe
    final = pd.DataFrame( )
    final = answer
    # Show the raw componenet output
    final.columns = [ 'close','mid','Daily Sine','Daily Cosine','Quarterly Sine','Quarterly Cosine','Annual Sine','Annual Cosine']
    #final.plot(legend=True, figsize=(5,3), xlabel='Minutes since '+firsttime+' UTC', colormap='turbo', title=symbol+': Generated Components ('+modeltype+')', lw=4)
    #plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
    #plt.savefig('pngs/'+symbol+modeltype+'raw.png',bbox_inches='tight',dpi=150)

    # Apply Weights and Denormalize

    final['close'] =  scaler.inverse_transform(np.array(final['close']*3).reshape(-1, 1))
    final['mid'] =  scaler.inverse_transform(np.array(final['mid']*2).reshape(-1, 1))
    final['Daily Sine'] =  scaler.inverse_transform(np.array(final['Daily Sine']*.05).reshape(-1, 1))
    final['Daily Cosine'] =  scaler.inverse_transform(np.array(final['Daily Cosine']*.05).reshape(-1, 1))
    final['Quarterly Sine'] =  scaler.inverse_transform(np.array(final['Quarterly Sine']*2).reshape(-1, 1))
    final['Quarterly Cosine'] =  scaler.inverse_transform(np.array(final['Quarterly Cosine']*2).reshape(-1, 1))
    final['Annual Sine'] =  scaler.inverse_transform(np.array(final['Annual Sine']*4).reshape(-1, 1))
    final['Annual Cosine'] =  scaler.inverse_transform(np.array(final['Annual Cosine']*4).reshape(-1, 1))
 
    # Show the weighted componenet output
    final.columns = [ 'close','mid','Daily Sine','Daily Cosine','Quarterly Sine','Quarterly Cosine','Annual Sine','Annual Cosine']
    #final.plot(legend=True, figsize=(5,3), xlabel='Hours since '+firsttime+' UTC', colormap='Dark2', title=symbol+': Weighted Components ('+modeltype+')', lw=4)
    #plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
    #plt.savefig('pngs/'+symbol+modeltype+'w8s.png',bbox_inches='tight',dpi=150)

    # Aggregate weighted predictions
    agg = np.ones(96)
    for i in range(96):
      agg[i] = np.mean(final.iloc[i])
    # Plot the aggregate prediction
    a=pd.DataFrame()
    a['agg']=pd.DataFrame(agg)
    a['Actual'] = realdf['close']
    a['Actual mid'] = realdf['mid']

    # Produce the Linear Baseline
    a['baseline'] = np.full(96,last)
    # Produce Cross Fade Table
    a['cross'] = np.linspace(0,1,96)
    a['rcross'] = np.linspace(1,0,96)
    # Cross-fade from the linear baseline
    a['basecross'] = a['baseline'] *( a['rcross'])
    a['outcross'] = (a['agg'] *  a['cross'])
    a['Predicted'] = a['outcross']+ a['basecross']
    a = a.drop('cross',axis=1)
    a = a.drop('rcross',axis=1)
    a = a.drop('basecross',axis=1)
    a = a.drop('baseline',axis=1)
    a = a.drop('agg',axis=1)
    a = a.drop('outcross',axis=1)
    a['Market Data'] = (a['Actual']+a['Actual mid'])/2
    a['diff'] = np.square(a['Predicted']-a['Market Data'])
    what = np.mean(a['diff'])/np.mean(a['Predicted'])
    print(what)
    a = a.drop('diff',axis=1)
    a = a.drop('Actual',axis=1)
    a = a.drop('Actual mid',axis=1)
    #a = a.drop('Market Data',axis=1)
    a.plot(legend=True, figsize=(5,3), xlabel='Hours since '+firsttime+' UTC', ylabel='$ (USD)', colormap='turbo', title=symbol+': Prediction vs Actual ('+modeltype+')', lw=3)
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%g'))
    plt.savefig('pngs/'+symbol+modeltype+'score.png', bbox_inches='tight', dpi=150)
    plt.close()

    print(symbol+' '+modeltype)

42035.6
267.1352543582023
BTC lstm
42035.6
265.0103406815307
BTC bidir
42035.6
362.5642894193245
BTC feedback
3129.74


KeyboardInterrupt: ignored



---



In [2]:
a

,agg,Actual,Actual mid,baseline,finalcross,ave
0,47077.597656,35950.31,35883.325,38185.93,38185.930000,35916.8175
1,50779.843750,35787.04,35623.090,38185.93,38318.497513,35705.0650
2,49968.312500,35426.97,35254.830,38185.93,38433.980158,35340.9000
3,49592.906250,35491.38,35057.045,38185.93,38546.150303,35274.2125
4,49532.925781,34831.76,35243.700,38185.93,38663.698243,35037.7300
...,...,...,...,...,...,...
91,49321.750000,41629.10,41537.355,38185.93,48852.873368,41583.2275
92,49321.363281,41505.51,41616.000,38185.93,48969.718020,41560.7550
93,49320.960938,41723.95,41595.990,38185.93,49086.539234,41659.9700
94,49320.546875,41602.55,41595.520,38185.93,49203.340382,41599.0350


In [ ]:
df

,time,close,mid
0,1642791600,38185.93,38268.565
1,1642788000,38289.79,38253.945
2,1642784400,38361.77,38516.005
3,1642780800,38728.17,38784.300
4,1642777200,38859.22,38577.505
...,...,...,...
91,1642464000,42386.09,42288.920
92,1642460400,42224.38,42181.680
93,1642456800,42099.51,41984.040
94,1642453200,41745.85,41893.070


In [ ]:
df.iloc[95]['mid']

0.5439793771271297

    # Tensorflow Routines for Time Series Data
    # https://www.tensorflow.org/tutorials/structured_data/time_series

    '''Martín Abadi, Ashish Agarwal, Paul Barham, Eugene Brevdo,
    Zhifeng Chen, Craig Citro, Greg S. Corrado, Andy Davis,
    Jeffrey Dean, Matthieu Devin, Sanjay Ghemawat, Ian Goodfellow,
    Andrew Harp, Geoffrey Irving, Michael Isard, Rafal Jozefowicz, Yangqing Jia,
    Lukasz Kaiser, Manjunath Kudlur, Josh Levenberg, Dan Mané, Mike Schuster,
    Rajat Monga, Sherry Moore, Derek Murray, Chris Olah, Jonathon Shlens,
    Benoit Steiner, Ilya Sutskever, Kunal Talwar, Paul Tucker,
    Vincent Vanhoucke, Vijay Vasudevan, Fernanda Viégas,
    Oriol Vinyals, Pete Warden, Martin Wattenberg, Martin Wicke,
    Yuan Yu, and Xiaoqiang Zheng.'''

    # TensorFlow: Large-scale machine learning on heterogeneous systems,
    # 2015. Software available from tensorflow.org.